In [0]:
import pandas as pd
from statsmodels.stats.power import TTestIndPower

In [0]:
%sql
select
  *
from
  workspace.money_mop.ref_category_amounts

In [0]:
df = _sqldf.toPandas()

df

In [0]:
effect_percent = 0.05

df['mde'] = df['mean_amount'] * effect_percent
df['effect_size'] = df['mde'] / df['std_amount']
df

In [0]:
analysis = TTestIndPower()
df['n_per_group'] = df['effect_size'].apply(
    lambda es: analysis.solve_power(effect_size=es, power=0.8, alpha=0.05)
).round().astype(int)
df

In [0]:
effect_size = 5 / 30 
analysis = TTestIndPower()
n_per_group = analysis.solve_power(effect_size=effect_size, power=0.8, alpha=0.05)
print(f"Transactions needed per group: {int(n_per_group)}")

In [0]:
spark.read.table('workspace.money_mop.silver_daily_transactions')

In [0]:
!pip install PyYAML

In [0]:
%restart_python